Als erstes wird der Datensatz erstellt und geladen

In [ ]:
import torch
import numpy as np
import pandas as pd

import torch.nn as nn
from MIFS import mutual_infxy
from torch.utils.data import Dataset

import torch.nn.functional as F

path_input = "./data/ANI_Training.Input"
path_target = "./data/ANI_Training.Label"

class ContestDataset(Dataset):
    def __init__(self, selected_channels):
        df = pd.read_csv(path_input, header=None) # (3250, 250)
        df_train = df.iloc[0:2900, :]
        # print("df_train: ", np.array(df_train).shape)

        target_train = pd.read_csv(path_target, header=None) # (3250, 1)
        target_train = target_train.iloc[0:2900, :]
        self.target = target_train

        data = np.array(df_train.T) # (250,  3250)
        data = data[selected_channels, ::]

        # Normalize each feature separately
        min_vals = np.min(data, axis=1, keepdims=True)
        max_vals = np.max(data, axis=1, keepdims=True)
        # print("min vals: ", min_vals)
        # print("min vals: ", max_vals)

        self.data = 2 * (data - min_vals) / (max_vals - min_vals) - 1

        # for i, channel in enumerate(self.data):
        #     print(i)
        #     MI = mutual_infxy(np.array(channel), np.array(self.target))
        #     print(MI)
        # time.sleep(100)
        
    def __len__(self):
        return self.data.shape[1] # number of examples in the dataset
    
    def __getitem__(self, idx):
        data = self.data[:, idx]
        target = self.target.iloc[idx, 0]
        return torch.tensor(data, dtype=torch.float32), torch.tensor(target, dtype=torch.float32)